## EU Driving regulation for truckers

The EU Regulation 561/2006 sets rules for driving time, breaks, and rest periods for drivers of goods vehicles over 3.5 tons. These regulations aim to improve road safety and protect drivers from excessive fatigue. This is a critical problem in the logistic industry that is actually handled by humans, manually defining the routes and breaks for the drivers. ...

Maximun Driving Time:
* Driving: 4.5 hours of continuous driving.
* Daily: 9 hours, extendable to 10 hours twice a week.
* Weekly: 56 hours.
* Bi-weekly: 90 hours in any two consecutive weeks.

Breaks:

* Driving: 45 minutes is mandatory (can be split into 15 minutes followed by 30 minutes) After 4.5 hours of continuous driving.
* Daily rest: 11 hours or reduced 9 hours (up to three times per week). After 9 hours of driving.
* Weekly rest: 45 hours or a reduced weekly rest period of at least 24 hours.



The tachograph records various driving and vehicle data, ensuring compliance with driving time, breaks, and rest period regulations. Information recorded by a tachograph includes:

* Driver's details: The driver's name, license number, and tachograph card number.
* Date and time: The device records date and time for all activities.
* Vehicle speed: The tachograph constantly tracks and records the vehicle's speed.
* Driving time: The device records the duration of driving sessions, including any breaks or rest periods.
* Distance traveled: The tachograph logs the distance covered during a driving session.
* Breaks and rest periods: The device records the duration and frequency of breaks and rest periods taken by the driver.
* Work mode: The tachograph differentiates between driving, working, availability, and rest modes.
* Events and faults: The device logs any events or faults, such as overspeeding, power supply interruptions, or sensor malfunctions.

# Example of a tachograph

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/driver1.csv")
status_translation = {
    'Pausa': 'Break',
    'Conduciendo': 'Driving',
    'Otro trabajo': 'Other work',
    'En espera': 'On standby'
}

# Replace the values in the Status column with the translated values
df['Status'] = df['Status'].map(status_translation)
df.head()

,Driver,DateStart,HourStart,HourEnd,Duration,Distance,Status,Profession,Vehicle,PlaceStart,PlaceEnd,LatitudeStart,LongitudeStart,LatitudeEnd,LongitudeEnd,MilometerStart,MilometerEnd
0,driver1,lun 2/01/17,17:59,18:13,0h 14m,0.028144,Break,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.327719,-3.784834,40.327719,-3.784502,"31,512","31,512"
1,driver1,lun 2/01/17,18:13,18:16,0h 03m,0.124866,Driving,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.327926,-3.784502,40.327926,-3.785975,"31,512","31,513"
2,driver1,lun 2/01/17,18:16,18:18,0h 02m,0.000509,Other work,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.329214,-3.785975,40.329214,-3.785981,"31,513","31,513"
3,driver1,lun 2/01/17,18:18,18:20,0h 02m,0.230232,Driving,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.328715,-3.785981,40.328715,-3.788697,"31,513","31,513"
4,driver1,lun 2/01/17,18:20,18:43,0h 23m,0.001611,Other work,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.328620,-3.788697,40.328620,-3.788678,"31,513","31,513"


# Solving it with UPF

In [3]:
%%capture
%pip install ..
%pip install ../unified-planning

In [4]:
from up_siadex import SIADEXEngine

import unified_planning as up
from unified_planning.shortcuts import *
from unified_planning.model.htn.hierarchical_problem import HierarchicalProblem, Task, Method
from unified_planning.io import PDDLReader
from unified_planning.io import PDDLWriter
from unified_planning.io.hpdl.hpdl_reader import HPDLReader
from unified_planning.io.hpdl.hpdl_writer import HPDLWriter
from unified_planning.engines.results import PlanGenerationResultStatus


In [5]:
env = up.environment.get_env()
env.factory.add_engine('siadex', __name__, "SIADEXEngine")

In [6]:
reader = HPDLReader()
problem = reader.parse_problem("../unified-planning/unified_planning/test/pddl/hpdl/jacob/domain.hpdl","../unified-planning/unified_planning/test/pddl/hpdl/jacob/problem.hpdl")

# The problem in UPF

In [7]:
task_translation = {
    "reset_counters": "Reset Counters",
    "wd": "Weekly Driving",
    "dd": "Daily Driving",
    "ignore_activity": "Ignore Activity",
    "ndd": "Normal Daily Driving",
    "edd": "Extended Daily Driving",
    "cdd": "Continuous Daily Driving",
    "cdd_unint": "Continuous Daily Driving Uninterrupted",
    "cdd_split_1": "Continuous Daily Driving Split 1",
    "cdd_split_2": "Continuous Daily Driving Split 2",
    "rest": "Rest",
    "dr": "Drive",
    "wr": "Weekly Rest",
    "b_t1": "Break Type 1",
    "b_t2": "Break Type 2",
    "b_t3": "Break Type 3",
    "a": "Activity",
    "process_a": "Process Activity",
    "d": "Drive",
    "o": "Other",
    "b": "Break",
    "i": "Interruption",
    "add_the_current_action_to_plan": "Add Current Action to Plan",
    "b_token": "Begin Token",
    "e_token": "End Token",
    "b_breaktype": "Begin Break Type",
    "e_breaktype": "End Break Type",
    "b_sequence": "Begin Sequence",
    "e_sequence": "End Sequence",
    "b_daytype": "Begin Day Type",
    "e_daytype": "End Day Type",
    "transport": "Transport",
    "transport-box": "Transport Box",
    "load": "Load",
    "unload": "Unload",
    "board": "Board",
    "debark": "Debark",
    "drive": "Drive",
    "drive-sequence": "Drive Sequence"
}

action_status = {
    "d_p": "Driving",
    "o_p": "Other Work",
    "b_p": "Break",
    "i_p": "Idle",
    "load_p": "Load Package",
    "unload_p": "Unload Package",
    "drive_p": "Drive to Destination",
    "drive_p_fixed_duration": "Drive with Fixed Duration",
    "refuel": "Refuel",
    "refuel-on-the-way": "Refuel on the Way",
    "d_suggested": "Suggested Driving",
    "b_suggested": "Suggested Break"
}

method_task_translation = {
    ('reset_counters-single', 'reset_counters'): "Reset Counters (Single)",
    ('wd-wr', 'wd'): "Weekly Driving (Weekly Rest)",
    ('wd-normal', 'wd'): "Weekly Driving (Normal)",
    ('wd-illegal_biweek', 'wd'): "Weekly Driving (Illegal Biweek)",
    ('wd-transport', 'wd'): "Weekly Driving (Transport)",
    ('wd-end', 'wd'): "Weekly Driving (End)",
    ('dd-ndd', 'dd'): "Daily Driving (Normal Daily Driving)",
    ('dd-edd', 'dd'): "Daily Driving (Extended Daily Driving)",
    ('dd-cdd', 'dd'): "Daily Driving (Continuous Daily Driving)",
    ('dd-rest_day', 'dd'): "Daily Driving (Rest Day)",
    ('dd-ignore_action', 'dd'): "Daily Driving (Ignore Action)",
    ('dd-end', 'dd'): "Daily Driving (End)",
    ('ignore_activity-activity', 'ignore_activity'): "Ignore Activity (Activity)",
    ('ignore_activity-unknown_break', 'ignore_activity'): "Ignore Activity (Unknown Break)",
    ('ndd-two_cdds', 'ndd'): "Normal Daily Driving (Two Continuous Daily Drivings)",
    ('ndd-unique', 'ndd'): "Normal Daily Driving (Unique)",
    ('edd-edd', 'edd'): "Extended Daily Driving (Extended Daily Driving)",
    ('cdd-uninterrupted', 'cdd'): "Continuous Daily Driving (Uninterrupted)",
    ('cdd-split', 'cdd'): "Continuous Daily Driving (Split)",
    ('cdd_unint-rest', 'cdd_unint'): "Continuous Daily Driving Uninterrupted (Rest)",
    ('cdd_unint-b_t1', 'cdd_unint'): "Continuous Daily Driving Uninterrupted (Break Type 1)",
    ('cdd_split_1-b_t2', 'cdd_split_1'): "Continuous Daily Driving Split 1 (Break Type 2)",
    ('cdd_split_2-end', 'cdd_split_2'): "Continuous Daily Driving Split 2 (End)",
    ('cdd_split_2-rd', 'cdd_split_2'): "Continuous Daily Driving Split 2 (RD)",
    ('cdd_split_2-b_t3', 'cdd_split_2'): "Continuous Daily Driving Split 2 (Break Type 3)",
    ('rest-daily', 'rest'): "Rest (Daily)",
    ('rest-weekly', 'rest'): "Rest (Weekly)",
    ('rest-illegal_daily', 'rest'): "Rest (Illegal Daily)",
    ('rest-illegal_weekly', 'rest'): "Rest (Illegal Weekly)",
    ('rest-fin_secuencia_entrada', 'rest'): "Rest (Fin Secuencia Entrada)",
    ('dr-dr_t4', 'dr'): "Drive (Drive Type 4)",
    ('dr-dr_t1', 'dr'): "Drive (Drive Type 1)",
    ('dr-dr_t2', 'dr'): "Drive (Drive Type 2)",
    ('dr-dr_t3', 'dr'): "Drive (Drive Type 3)",
    ('wr-wr_t1', 'wr'): "Weekly Rest (Weekly Rest Type 1)",
    ('wr-wr_t2', 'wr'): "Weekly Rest (Weekly Rest Type 2)",
    ('b_t1-b_t1', 'b_t1'): "Break Type 1 (Break Type 1)",
    ('b_t2-b_t2', 'b_t2'): "Break Type 2 (Break Type 2)",
    ('b_t3-b_t3', 'b_t3'): "Break Type 3 (Break Type 3)",
    ('a-recurrir', 'a'): "A (Recurrir)",
    ('a-transport', 'a'): "A (Transport)",
    ('a-pause_considered_break', 'a'): "A (Pause Considered Break)",
    ('a-fin_secuencia_entrada', 'a'): "A (Fin Secuencia Entrada)",
    ('a-fallar', 'a'): "A (Fallar)",
    ('process_a-dxx', 'process_a'): "Process A (DXX)",
    ('process_a-oxx', 'process_a'): "Process A (OXX)",
    ('process_a-b_t0', 'process_a'): "Process A (Break Type 0)",
    ('process_a-ixx', 'process_a'): "Process A (IXX)",
    ('d-modo_reconocer', 'd'): "D (Modo Reconocer)",
    ('d-modo_generar', 'd'): "D (Modo Generar)",
    ('o-modo_reconocer', 'o'): "O (Modo Reconocer)",
    ('b-modo_reconocer', 'b'): "B (Modo Reconocer)",
    ('b-modo_generar', 'b'): "B (Modo Generar)",
    ('b-fin_secuencia_entrada', 'b'): "B (Fin Secuencia Entrada)",
    ('i-modo_reconocer', 'i'): "I (Modo Reconocer)",
    ('add_the_current_action_to_plan-type_driving', 'add_the_current_action_to_plan'): "Add the Current Action to Plan (Type Driving)",
    ('add_the_current_action_to_plan-type_other', 'add_the_current_action_to_plan'): "Add the Current Action to Plan (Type Other)",
    ('add_the_current_action_to_plan-type_break', 'add_the_current_action_to_plan'): "Add the Current Action to Plan (Type Break)",
    ('add_the_current_action_to_plan-type_idle', 'add_the_current_action_to_plan'): "Add the Current Action to Plan (Type Idle)",
    ('b_token-unico', 'b_token'): "B Token (Unico)",
    ('e_token-unico', 'e_token'): "E Token (Unico)",
    ('b_breaktype-unico', 'b_breaktype'): "B Break Type (Unico)",
    ('e_breaktype-unico', 'e_breaktype'): "E Break Type (Unico)",
    ('b_sequence-unico', 'b_sequence'): "B Sequence (Unico)",
    ('e_sequence-unico', 'e_sequence'): "E Sequence (Unico)",
    ('b_daytype-unico', 'b_daytype'): "B Day Type (Unico)",
    ('e_daytype-unico', 'e_daytype'): "E Day Type (Unico)",
    ('transport-delivery', 'transport'): "Transport (Delivery)",
    ('transport-finished', 'transport'): "Transport (Finished)",
    ('transport-box-boxatdestination', 'transport-box'): "Transport Box (Box at Destination)",
    ('transport-box-unloadbox', 'transport-box'): "Transport Box (Unload Box)",
    ('transport-box-Doxanddriveratcity', 'transport-box'): "Transport Box (DOX and Driver at City)",
    ('transport-box-continuingsplitdriving', 'transport-box'): "Transport Box (Continuing Split Driving)",
    ('transport-box-boxalreadyloaded', 'transport-box'): "Transport Box (Box Already Loaded)",
    ('transport-box-Driverinothercity', 'transport-box'): "Transport Box (Driver in Other City)",
    ('load-same_destination', 'load'): "Load (Same Destination)",
    ('load-any', 'load'): "Load (Any)",
    ('load-end', 'load'): "Load (End)",
    ('unload-unload', 'unload'): "Unload (Unload)",
    ('unload-end', 'unload'): "Unload (End)",
    ('board-normal', 'board'): "Board (Normal)",
    ('board-weight_surpassed', 'board'): "Board (Weight Surpassed)",
    ('debark-unique', 'debark'): "Debark (Unique)",
    ('drive-no-refuel', 'drive'): "Drive (No Refuel)",
    ('drive-refuel', 'drive'): "Drive (Refuel)",
    ('drive-refuel-mid-way', 'drive'): "Drive (Refuel Mid-Way)",
    ('drive-sequence-break2', 'drive-sequence'): "Drive Sequence (Break 2)",
    ('drive-sequence-no_break', 'drive-sequence'): "Drive Sequence (No Break)",
    ('drive-sequence-break', 'drive-sequence'): "Drive Sequence (Break)"
}

# Domain

In [31]:
def create_tree(task_translation, method_task_translation):
    tree = {}

    for (method, task), translation in method_task_translation.items():
        task_name = task_translation[task]
        if task_name not in tree:
            tree[task_name] = {}
        tree[task_name][method] = translation

    return tree


def print_tree(tree):
    print(f"___________________\n")
    for task_name, methods in tree.items():
        print(f"Task: [{task_name}]:")
        print(f"Methods:")
        for method, translation in methods.items():
            print(f"  - {translation}")
        print(f"___________________\n")
    print(f"Actions:")
    for action,translation in action_status.items():
        print(f"- {translation}")


tree = create_tree(task_translation, method_task_translation)
print_tree(tree)

___________________

Task: [Reset Counters]:
Methods:
  - Reset Counters (Single)
___________________

Task: [Weekly Driving]:
Methods:
  - Weekly Driving (Weekly Rest)
  - Weekly Driving (Normal)
  - Weekly Driving (Illegal Biweek)
  - Weekly Driving (Transport)
  - Weekly Driving (End)
___________________

Task: [Daily Driving]:
Methods:
  - Daily Driving (Normal Daily Driving)
  - Daily Driving (Extended Daily Driving)
  - Daily Driving (Continuous Daily Driving)
  - Daily Driving (Rest Day)
  - Daily Driving (Ignore Action)
  - Daily Driving (End)
___________________

Task: [Ignore Activity]:
Methods:
  - Ignore Activity (Activity)
  - Ignore Activity (Unknown Break)
___________________

Task: [Normal Daily Driving]:
Methods:
  - Normal Daily Driving (Two Continuous Daily Drivings)
  - Normal Daily Driving (Unique)
___________________

Task: [Extended Daily Driving]:
Methods:
  - Extended Daily Driving (Extended Daily Driving)
___________________

Task: [Continuous Daily Driving]:


# Solving with Siadex

In [27]:
with OneshotPlanner(name='siadex') as p:
    result = p.solve(problem)
    # print(result)
    if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'{p.name()} found a valid plan!')
        print(f'The plan is: \n')
        print(f"_"*50)
        for i,a in enumerate(result.plan.timed_actions):
            action_name = a[1].action.name
            action_params = [p.name for p in a[1].action.parameters]
            if a[1].action.name in action_status:
                action_name = action_status[action_name]
            print(f"""Start: {a[0]} - End: {a[0] + a[2]} \nAction: {action_name}""")
            for param,value in list(zip(action_params,a[1].actual_parameters)):
                print(f"    - {param}: {value}")    
                
            print(f"_"*50)
    else:
        print('No plan found!')

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_390/2884070800.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

SIADEX found a valid plan!
The plan is: 

__________________________________________________
Start: 0 - End: 5 
Action: Break
    - d: driver
    - dur: 5
    - tkctxt: b_t0
    - drivctxt: split_1
    - seqctxt: first
    - dayctxt: ndd
    - weekcount: 1
    - daycount: 1
    - legalctxt: yes
__________________________________________________
Start: 5 - End: 56 
Action: Driving
    - d: driver
    - dur: 51
    - tkctxt: a
    - drivctxt: split_1
    - seqctxt: first
    - dayctxt: ndd
    - weekcount: 1
    - daycount: 1
    - legalctxt: yes
__________________________________________________
Start: 56 - End: 193 
Action: Break
    - d: driver
    - dur: 137
    - tkctxt: b_t2
    - drivctxt: split_1
    - seqctxt: first
    - dayctxt: ndd
    - we